# 강남구 (1080)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 6일
- 편집자 : 이준석
- 편집일자 : 2020년 9월 6일

In [14]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [11]:
df = pd.read_csv("200906one_shot.csv")
soup = BeautifulSoup(df['html'][0], 'html.parser')

### 2. Regex Parsing 하기

In [40]:
# 너무 길어서 두 개로 나누었다. (가시적으로 표현하자.)
regex = 'tle\"\>\#(?P<num>.+?)\s확진자\\t\((?P<rsd>.+?)\).+?\/\s?(?P<dat>.+?)\<\/div.+?<li>감염경로\s\:\s?'
regex += '(?P<pth>.+?)\<\/li\>\s\<li\>(증상발현일\s\:\s)?(?P<sym>.+?)\<\/li>(.+?조치\s\:\s(?P<atn>.+?)\<\/li\>)?'

rows = soup.select('#tab-1 .acodList')
infos = [re.search(regex, str(r)) for r in rows]

### 3. 저장할 DataFrame 미리 만들기

In [41]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

In [42]:
new_route = pd.read_html(str(rows[4].select('table')[0]))[0]

### 4. 귀찮은 전처리해주기

In [43]:
for i in tqdm(infos):
    
    # regex 처리가 안 되는 애들은 어차피 정보가 없으므로 None 값임.
    if i is not None:
        
        num = '1080' + '0' * (5-len(i.group('num'))) + i.group('num')
        syy = i.group('sym') != '증상 없음'
        syd = i.group('sym') * syy
        dch = ''; adt = '' # 정보가 없으면 이렇게 처리하자
        rsd, cnd, pth, sym, atn = i.group('rsd'), i.group('dat'), i.group('pth'), i.group('sym'), i.group('atn')
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt)], columns = pat_col)
        
        patient = pd.concat([patient, new_pat])
        
        new_route = pd.read_html(str(rows[4].select('table')[0]))[0]
        new_route['rgl'] = new_route.지역 + new_route.위치
        new_route['frm'] = ''; new_route['msk'] = ''
        new_route['num'] = num
        new_route['ord'] = range(1, len(new_route)+1)
        new_route['dat'] = dat
        new_route = new_route.drop(['지역', '위치'], axis = 1)
        new_route.columns = ['rgt', 'exd', 'mob', 'ste', 'rgl', 'frm', 'msk', 'num', 'ord', 'dat']
        new_route = new_route.loc[:,rou_col]
        
        route = pd.concat([route, new_route])

100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:01<00:00, 73.95it/s]


### 5. 결과 잘 나왔나 확인하기

In [44]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt
0,108000213,2020-09-06,확진자 접촉,True,9.4,2020-09-06,세곡동,None,,
0,108000212,2020-09-06,확진자 접촉,False,,2020-09-05,개포동,None,,
0,108000211,2020-09-06,확진자 접촉,True,8.29,2020-09-05,역삼동,None,,
0,108000210,2020-09-06,확진자 접촉,True,8.31,2020-09-05,개포동,None,,
0,108000209,2020-09-06,미상,True,9.3,2020-09-04,개포동,None,,
0,108000208,2020-09-06,미상,False,,2020-09-04,개포동,None,,
0,108000207,2020-09-06,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000206,2020-09-06,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000205,2020-09-06,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000204,2020-09-06,미상,True,8.30,2020-09-04,개포동,None,,


In [45]:
route.head(3)

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,108000213,1,2020-09-06,개포동대청역 1번 출구,카페,,- 9.1(화) 16:15,,완료,도보
1,108000213,2,2020-09-06,개포동대청역 1번 출구,편의점,,- 9.2(화) 12:00~12:20,,완료,NaN
2,108000213,3,2020-09-06,개포동대청역 1번 출구,편의점,,- 9.2(화) 18:43,,완료,NaN
